In [3]:
!pip install tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.0 MB/s eta 0:00:00


In [5]:
!pip uninstall -y fitz
!pip install PyMuPDF


Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Successfully uninstalled fitz-0.0.1.dev2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.2 MB/s eta 0:00:00


In [3]:


import fitz
import re
import nltk
from nltk.tokenize import sent_tokenize


nltk.download('punkt')
nltk.download('punkt_tab')



pdf_path = "/content/A_Brief_Introduction_To_AI.pdf"
doc = fitz.open(pdf_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Basic preprocessing the data by removing extra lines,spaces,page numbers,special symbols and headers

In [4]:
all_text = []

for page_num, page in enumerate(doc, start=1):
    text = page.get_text("text")


    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    text = re.sub(r'(Page\s*\d+)|(\b\d+\b(?:\s*/\s*\d+)?)', '', text, flags=re.IGNORECASE)

    text = re.sub(r'[•◦▪●]', '', text)

    if page_num == 1:
        first_page_sample = text[:100]
    else:
        text = text.replace(first_page_sample, '')

    all_text.append(text.strip())

doc.close()

Combine text from all pages

In [5]:
clean_text = " ".join(all_text)
clean_text = clean_text.strip()

splitting the data into chunks of size 512

In [6]:
sentences = sent_tokenize(clean_text)
chunks = []
current_chunk = []
current_length = 0
max_tokens = 512

for sentence in sentences:
    words = sentence.split()
    sentence_length = len(words)

    # If adding this sentence makes chunk too big, start a new one
    if current_length + sentence_length > max_tokens:
        chunks.append(" ".join(current_chunk))
        current_chunk = [sentence]
        current_length = sentence_length
    else:
        current_chunk.append(sentence)
        current_length += sentence_length


if current_chunk:
    chunks.append(" ".join(current_chunk))

example printing 1st chunk

In [7]:
print(chunks[0][:1000])

A Brief Introduction to Artificial Intelligence What is AI and how is it going to shape the future By Dibbyo Saha, Undergraduate Student, Computer Science, Ryerson University What is Artificial Intelligence? Image by Gerd Altmann from Pixabay Generally speaking, Artificial Intelligence is a computing concept that helps a machine think and solve complex problems as we humans do with our intelligence. For example, we perform a task, make mistakes and learn from our mistakes (At least the wise ones of us do!). Likewise, an AI or Artificial Intelligence is supposed to work on a problem, make some mistakes in solving the problem and learn from the problems in a self-correcting manner as a part of its self-improvement. Or in other words, think of this like playing a game of chess. Every bad move you make reduces your chances of winning the game. So, every time you lose against your friend, you try remembering the moves you made which you shouldn’t have and apply that knowledge in your next g

TASK -3

In [8]:
!pip install sentence-transformers faiss-cpu chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.2 MB/s eta 

 Vector Databases & Semantic Search on PDF Chunks

In [10]:



from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
import time
from sklearn.feature_extraction.text import TfidfVectorizer


Creating embeddings and normalizing

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, normalize_embeddings=True)
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (4, 384)


Store embeddings in FAISS

cosine similarity

In [12]:

dim = embeddings.shape[1]
index_cosine = faiss.IndexFlatIP(dim)
index_cosine.add(embeddings)





dot similarity

In [13]:
embeddings_dot = model.encode(chunks, normalize_embeddings=False)
index_dot = faiss.IndexFlatIP(dim)
index_dot.add(embeddings_dot)

Store in ChromaDB

In [15]:



chroma_client = chromadb.Client()
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = chroma_client.get_collection(name="pdf_chunks", embedding_function=embedding_fn)

print("Connected to existing collection.")


Connected to existing collection.


In [16]:

query = "How do machines understand human language?"
print(f"\n Query: {query}\n")


start = time.time()
query_vec = model.encode([query], normalize_embeddings=True)
scores, idx = index_cosine.search(query_vec, k=3)
end = time.time()
print(f" FAISS (Cosine) | Time: {end - start:.4f}s")
for i, score in zip(idx[0], scores[0]):
    print(f"{round(score,3)} : {chunks[i][:150]}...")
print("-" * 60)


start = time.time()
query_vec = model.encode([query], normalize_embeddings=False)
scores, idx = index_dot.search(query_vec, k=3)
end = time.time()
print(f" FAISS (Dot Product) | Time: {end - start:.4f}s")
for i, score in zip(idx[0], scores[0]):
    print(f"{round(score,3)} : {chunks[i][:150]}...")
print("-" * 60)


start = time.time()
results = collection.query(query_texts=[query], n_results=3)
end = time.time()
print(f" Chroma Semantic Search | Time: {end - start:.4f}s")
for doc, dist in zip(results["documents"][0], results["distances"][0]):
    print(f"{round(dist,3)} : {doc[:150]}...")
print("-" * 60)


def hybrid_search(query, chunks, embeddings, weight=0.5):
    query_vec = model.encode([query], normalize_embeddings=True)
    semantic_scores = np.dot(embeddings, query_vec.T).flatten()

    vectorizer = TfidfVectorizer().fit(chunks + [query])
    tfidf_matrix = vectorizer.transform(chunks + [query])
    keyword_scores = (tfidf_matrix[:-1] @ tfidf_matrix[-1].T).toarray().flatten()

    combined = weight * semantic_scores + (1 - weight) * keyword_scores
    top_idx = combined.argsort()[::-1][:3]
    return [(chunks[i], combined[i]) for i in top_idx]

print("\nHybrid Search Results (Semantic + Keyword):")
for doc, score in hybrid_search(query, chunks, embeddings):
    print(f"{round(score,3)} : {doc[:150]}...")





 Query: How do machines understand human language?

 FAISS (Cosine) | Time: 0.0299s
0.47200000286102295 : Unlike the conventional “bips and bops”, a good AI will simulate the complicated and intuitive sense of thinking and problem-solving abilities of the ...
0.38999998569488525 : For example, if you give the machine an image of a basketball and it identifies the basketball as a tennis ball or something else, you give a negative...
0.3409999907016754 : A Brief Introduction to Artificial Intelligence What is AI and how is it going to shape the future By Dibbyo Saha, Undergraduate Student, Computer Sci...
------------------------------------------------------------
 FAISS (Dot Product) | Time: 0.0167s
0.47200000286102295 : Unlike the conventional “bips and bops”, a good AI will simulate the complicated and intuitive sense of thinking and problem-solving abilities of the ...
0.38999998569488525 : For example, if you give the machine an image of a basketball and it identifies the basketba